In [ ]:
]activate ../..

In [ ]:
using Gen
using GenWorldModels

In [ ]:
include("../main.jl")
AI = AudioInference

In [ ]:
using .AudioInference: vis_wave, generate_initial_tr, generate_scene, args

In [ ]:
ground_truth, _ = generate(AudioInference.generate_scene, args)
println("Num sources: ", ground_truth[:kernel => :n_tones], "; score = ", get_score(ground_truth))
vis_wave(ground_truth)

In [ ]:
initial_tr, weight = AudioInference.generate_initial_tr(ground_truth, num_sources=0)
vis_wave(initial_tr)
println("Num sources: ", initial_tr[:kernel => :n_tones], "; weight = ", weight)

In [ ]:
(noises_to_add, tones_to_add) = AudioInference.analyze_errorgram(initial_tr)

In [ ]:
AudioInference.plot_gtg(AudioInference.error_gram(initial_tr))

In [ ]:
function img_with_tones(img, tones, val=100)
  img = copy(img)
  for tone in tones
    add_tone_to_img!(img, tone; val=val)
  end
  return img
end

function add_tone_to_img!(img, tone; val=100)
  (region, erb) = tone
  yval = Int(floor(AudioInference.pos_for_erb_val(erb)))
  for i=region[1]:region[2]
    img[yval, i] = val
  end
end

In [ ]:
function add_vert_bars(img, xvals; val=100)
  img = copy(img)
  for xval in xvals
    add_vert_bar!(img, xval, val=val)
  end
  img
end
function add_vert_bar!(img, xval; val=100)
    img[:, xval] .= val
end

In [ ]:
minimum(AudioInference.error_gram(initial_tr))

In [ ]:
AudioInference.plot_gtg(img_with_tones(AudioInference.error_gram(initial_tr), tones_to_add))

In [ ]:
gram = AudioInference.error_gram(initial_tr)
cps = AI.get_changepoints(AI.normalize(sum(gram, dims=1)); threshold_frac=0.05)

In [ ]:
AI.plot_gtg(add_vert_bars(AI.error_gram(initial_tr), cps))

In [ ]:
filter = [
  -0.2 -0.5 0 0.5 0.2;
  -0.4 -1.0 0 1.0 0.4;
  -0.2 -0.5 0 0.5 0.2
]
convd = AI.conv(gram, filter)
AI.plot_gtg(1/2*convd .+ 50)

In [ ]:
AI.plot_gtg(convd .* (convd .> 30))

In [ ]:
AI.plot_gtg(abs.(convd .* (convd .< -30)))

In [ ]:
startsimg = convd .< -30
endsimg = convd .> 30

In [ ]:
AI.plot_gtg(100*(startsimg - endsimg) .+ 50)

In [ ]:
]add ImageMorphology

In [ ]:
using ImageMorphology

In [ ]:
@time component_boxes(label_components(startsimg))

In [ ]:
function draw_rect!(img, ((x1, y1), (x2, y2)); color=100)
  for x=x1:min(x2, size(img)[1])
    img[x, min(y1, size(img)[2])] = color
    img[x, min(y2, size(img)[2])] = color
  end
  for y=y1:min(y2, size(img)[2])
    img[min(x1, size(img)[1]), y] = color
    img[min(x2, size(img)[1]), y] = color
  end
end

In [ ]:
function img_with_rects(img, rects; color=100)
  img = copy(img)
  for rect in rects
    draw_rect!(img, rect; color=color)
  end
  return img
end

In [ ]:
rects = component_boxes(label_components(startsimg))
AI.plot_gtg(img_with_rects(gram, rects[2:end]))

In [ ]:
struct VertSegment
  x::Int
  ymin::Int
  ymax::Int
end
startseg(((ymin, xmin), (ymax, xmax))) = VertSegment(xmin, ymin, ymax)
endseg(((ymin, xmin), (ymax, xmax))) = VertSegment(xmax, ymin, ymax)

In [ ]:
function add_vert_segs(img, segs; val=100)
  img = copy(img)
  for seg in segs
    add_vert_seg!(img, seg, val=val)
  end
  img
end
function add_vert_seg!(img, seg; val=100)
    for y=seg.ymin:min(seg.ymax, size(img)[1])
      img[y, seg.x] = val
  end
end

In [ ]:
startrects = component_boxes(label_components(startsimg))[2:end]
endrects = component_boxes(label_components(endsimg))[2:end]
startsegs = map(startseg, startrects)
endsegs = map(startseg, endrects)

img_with_starts_ends = add_vert_segs( add_vert_segs(gram, startsegs, val=100), endsegs, val=-100 )

AI.plot_gtg(1/2*img_with_starts_ends .+ 50)

In [ ]:
could_be_noise(seg) = seg.ymax - seg.ymin > 64/3
could_be_tone(seg) = let diff = seg.ymax - seg.ymin; diff >= 5 && diff <= 64/2; end
function possible_tone_noise_starts(startsegs)
  noisesegs = []
  tonesegs = []
  for seg in startsegs
    if could_be_noise(seg)
      push!(noisesegs, seg)
    end
    if could_be_tone(seg)
      push!(tonesegs, seg)
    end
  end
  return (noisesegs, tonesegs)
end

In [ ]:
noisesegs, tonesegs = possible_tone_noise_starts(startsegs)

In [ ]:
possible_endsegs(startseg, endsegs) = (seg for seg in endsegs if could_end(startseg, seg))
function could_end(startseg, endseg)
    endseg.x > startseg.x && overlap((startseg.ymin, startseg.ymax), (endseg.ymin, endseg.ymax)) > 0
end
function overlap((a, b), (c, d))
    if a > c
        return overlap((c, d), (a, b))
    end
    if b < c
        return 0
    end
    if d < b
        return (d - c) + 1
    end
    return b - c + 1
end

In [ ]:
collect(possible_endsegs(startsegs[2], endsegs))

In [ ]:
function tonerect(startseg, endseg)
  if endseg.ymax - endseg.ymin > 50
      return ((startseg.ymin, startseg.x), (startseg.ymax, endseg.x))
  end
  ymin = min(startseg.ymin, endseg.ymin)
  ymax = max(startseg.ymax, endseg.ymax)
  return ((ymin, startseg.x), (ymax, endseg.x))
end

In [ ]:
function noiserect(startseg, endseg)
  return ((1, startseg.x), (64, endseg.x))
end

In [ ]:
segpairs(startsegs) = Iterators.flatten(
  ((startseg, endseg) for endseg in possible_endsegs(startseg, endsegs))
  for startseg in startsegs
)

In [ ]:
rects = [map(x -> tonerect(x...), segpairs(tonesegs))...,  map(x -> noiserect(x...), segpairs(noisesegs))...]

In [ ]:
AI.plot_gtg(img_with_rects(gram, rects))

In [ ]:
include("../detector.jl")

In [ ]:
AI.plot_gtg(img_with_rects(gram, Iterators.flatten(Detector.get_noise_tone_rects(gram; threshold=0.5))))

In [ ]:
ground_truth2, _ = generate(AudioInference.generate_scene, args, choicemap((:kernel => :n_tones, 4)))
println("Num sources: ", ground_truth2[:kernel => :n_tones], "; score = ", get_score(ground_truth2))
vis_wave(ground_truth2)

In [ ]:
initial_tr, weight = AudioInference.generate_initial_tr(ground_truth2, num_sources=0)
gram = AI.error_gram(initial_tr)
AI.plot_gtg(img_with_rects(gram, Iterators.flatten(Detector.get_noise_tone_rects(gram; threshold=0.4))))